Gabriel Avinaz\
Week 7\
4/26/23

## Assignment 7.1
### Part a

In [52]:
import os
import shutil
import pandas as pd

In [53]:
path = "data"
if not os.path.exists(path):
    os.makedirs(path)
    shutil.copy("../assignment03/results/routes.parquet", path)

In [54]:
routes_df = pd.read_parquet("data/routes.parquet", engine='pyarrow')
routes_df = pd.json_normalize(routes_df["Flight Info"])
routes_df.sample(10)

,codeshare,equipment,airline.active,airline.airline_id,airline.alias,airline.callsign,airline.country,airline.iata,airline.icao,airline.name,...,src_airport.icao,src_airport.latitude,src_airport.longitude,src_airport.name,src_airport.source,src_airport.timezone,src_airport.type,src_airport.tz_id,dst_airport,src_airport
54155,False,"[736, 32A]",True,4870,TAP Air Portugal,TUNAIR,Tunisia,TU,TAR,Tunisair,...,DTMB,35.758099,10.754700,Monastir Habib Bourguiba International Airport,OurAirports,1.0,airport,Africa/Tunis,NaN,NaN
8482,False,"[E90, 319, 320, 763, E75]",True,330,ANA All Nippon Airways,AIR CANADA,Canada,AC,ACA,Air Canada,...,CYYZ,43.677200,-79.630600,Lester B. Pearson International Airport,OurAirports,-5.0,airport,America/Toronto,NaN,NaN
248,False,[AT7],False,1729,SN Brussels Airlines,CHANGAN,China,2Z,CGN,Changan Airlines,...,SBRP,-21.136389,-47.776669,Leite Lopes Airport,OurAirports,-3.0,airport,America/Sao_Paulo,NaN,NaN
21840,False,[320],True,2009,CSA Czech Airlines,DELTA,United States,DL,DAL,Delta Air Lines,...,KSLC,40.788399,-111.977997,Salt Lake City International Airport,OurAirports,-7.0,airport,America/Denver,NaN,NaN
23028,False,[320],True,837,Pulkovo Aviation Enterprise,SHAMROCK,Ireland,EI,EIN,Aer Lingus,...,EICK,51.841301,-8.491110,Cork Airport,OurAirports,0.0,airport,Europe/Dublin,NaN,NaN
57589,True,[CR7],True,5209,TWA,UNITED,United States,UA,UAL,United Airlines,...,KSAT,29.533701,-98.469803,San Antonio International Airport,OurAirports,-6.0,airport,America/Chicago,NaN,NaN
5838,True,[320],True,24,\N,AMERICAN,United States,AA,AAL,American Airlines,...,EGLL,51.470600,-0.461941,London Heathrow Airport,OurAirports,0.0,airport,Europe/London,NaN,NaN
48395,False,[320],True,4304,Qantas Airways,AIR AZORES,Portugal,S4,RZO,SATA International,...,LPPT,38.781300,-9.135920,Humberto Delgado Airport (Lisbon Portela Airport),OurAirports,0.0,airport,Europe/Lisbon,NaN,NaN
50932,False,[319],True,130,\N,AEROFLOT,Russia,SU,AFL,Aeroflot Russian Airlines,...,ULLI,59.800301,30.262501,Pulkovo Airport,OurAirports,3.0,airport,Europe/Moscow,NaN,NaN
61585,False,[321],True,5309,TWA,VIET NAM AIRLINES,Vietnam,VN,HVN,Vietnam Airlines,...,VVPC,13.955000,109.042000,Phu Cat Airport,OurAirports,7.0,airport,Asia/Saigon,NaN,NaN


In [55]:
def key_gen(row):
    return str(row['src_airport.iata']) + str(row['dst_airport.iata']) + str(row['airline.iata'])

In [56]:
routes_df["key"] = routes_df.apply(key_gen, axis=1)
routes_df[['src_airport.iata', 'dst_airport.iata', 'airline.iata', 'key']].sample(10)

,src_airport.iata,dst_airport.iata,airline.iata,key
20311,BJX,ATL,DL,BJXATLDL
50111,PMI,CPH,SK,PMICPHSK
50113,PMI,OSL,SK,PMIOSLSK
18568,HGH,XIY,CZ,HGHXIYCZ
20640,DTW,LHR,DL,DTWLHRDL
1543,PMI,TXL,4U,PMITXL4U
16058,CAN,GYS,CA,CANGYSCA
62530,ENU,ABV,W3,ENUABVW3
55284,OPO,CDG,U2,OPOCDGU2
34573,HND,CTS,JL,HNDCTSJL


In [57]:
partitions = (
    ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
    ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
    ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
    ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
)

def kv_key_gen(row):
    for tuple in partitions:
        first_char = row["key"][:1]
        if first_char >= tuple[0] and first_char <= tuple[1]:
            if tuple[0] == tuple[1]:
                return tuple[0]
            else:
                return tuple[0] + '-' + tuple[1]

In [58]:
routes_df["kv_key"] = routes_df.apply(kv_key_gen, axis=1)
routes_df[['src_airport.iata', 'dst_airport.iata', 'airline.iata', 'key', 'kv_key']].sample(10)

,src_airport.iata,dst_airport.iata,airline.iata,key,kv_key
9858,ORY,PTP,AF,ORYPTPAF,O-P
26248,BRU,ALC,FR,BRUALCFR,B
18884,NNG,HGH,CZ,NNGHGHCZ,N
32812,ORD,DUS,IB,ORDDUSIB,O-P
59418,JFK,BOG,US,JFKBOGUS,I-J
10927,CUL,MTY,AM,CULMTYAM,C-D
35091,BDO,JOG,JT,BDOJOGJT,B
19447,YCU,CKG,CZ,YCUCKGCZ,Y-Z
48872,JNB,MRU,SA,JNBMRUSA,I-J
17668,PTY,CUN,CM,PTYCUNCM,O-P


In [59]:
routes_df.to_parquet('results/kv/', partition_cols=['kv_key'])

In [60]:
routes_df.isnull().sum()

codeshare                     0
equipment                     0
airline.active                0
airline.airline_id            0
airline.alias                 0
airline.callsign              0
airline.country               0
airline.iata                  0
airline.icao                  0
airline.name                  0
dst_airport.airport_id      488
dst_airport.altitude        488
dst_airport.city            488
dst_airport.country         488
dst_airport.dst             488
dst_airport.iata            488
dst_airport.icao            488
dst_airport.latitude        488
dst_airport.longitude       488
dst_airport.name            488
dst_airport.source          488
dst_airport.timezone        488
dst_airport.type            488
dst_airport.tz_id           488
src_airport.airport_id      483
src_airport.altitude        483
src_airport.city            483
src_airport.country         483
src_airport.dst             483
src_airport.iata            483
src_airport.icao            483
src_airp

### Part b

In [61]:
import hashlib

def hash_key(key):
    m = hashlib.sha256()
    m.update(str(key).encode('utf-8'))
    return m.hexdigest()

In [62]:
routes_df["hashed"] = routes_df.apply(lambda x: hash_key(x["key"]), axis=1)
routes_df[['src_airport.iata', 'dst_airport.iata', 'airline.iata', 'key', 'hashed']].sample(10)

,src_airport.iata,dst_airport.iata,airline.iata,key,hashed
60576,BIO,IBZ,UX,BIOIBZUX,898a50f70d6b192ed05577a8e68762242ba2865de33a80...
41484,CTU,XIY,MU,CTUXIYMU,bbc4f2f62aa6085d4e2ca76eb7de857679078948b3be3e...
6476,PHL,PBI,AA,PHLPBIAA,7cdae514c2c5506b2e86ce62bb0ccfe9a7e89db2f39ca8...
42550,XMN,NGB,MU,XMNNGBMU,3858e2fab8da655823a1657d62d420416a7de4d680fcce...
9371,CDG,NRT,AF,CDGNRTAF,efbadb561c0bb79c97713664a98693e85b1a5bcbcea88b...
15967,POS,FLL,BW,POSFLLBW,c6f18c5cec8758f526d8fca14e205a81236d8122ec7da2...
44239,BEG,ATH,OA,BEGATHOA,d1d1a0c6e7847f0142cd00d79ca0e2d90b0da63773010a...
15993,KIX,PUS,BX,KIXPUSBX,31bb38abf5f5eec009769e1e6210bc965dcad20e10e346...
33886,CGO,HAK,JD,CGOHAKJD,bdbf88317b55a2dd693b13ef358fb7ee8d9d87e89e0d90...
54734,EDI,STN,U2,EDISTNU2,72092f7f7168e3da0d4100f61c99c0f7d634da93b9acbf...


In [63]:
routes_df["hash_key"] = routes_df.apply(lambda x: x["hashed"][0].upper(), axis=1)

In [64]:
routes_df.to_parquet('results/hash/', partition_cols=['hash_key'])